In [77]:
# Importing Functions
from pyspark.sql.functions import isnan, when, count, col, isnull, concat, lit,to_timestamp, desc

In [3]:
# Creating Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("park").getOrCreate()

In [4]:
# Reading data from CSV file and storing values in dataframe
# Download Data file from kaggle(https://www.kaggle.com/new-york-city/nyc-parking-tickets/data)
df = spark.read.format("com.databricks.spark.csv").option("header","true").option("inferSchema", "true").load("/common_folder/nyc_parking/Parking_Violations_Issued_-_Fiscal_Year_2017.csv")

In [5]:
#t verify if data frame is created
df.show(2)

+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
|Summons Number|Plate ID|Registration State|         Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Violation Precinct|Issuer Precinct|Violation Time|
+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
|    5092469481| GZH7067|                NY|2016-07-10 00:00:00|             7|             SUBN|       TOYOT|                 0|              0|         0143A|
|    5092451658| GZH7067|                NY|2016-07-08 00:00:00|             7|             SUBN|       TOYOT|                 0|              0|         0400P|
+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
only showing top 2 rows



In [6]:
#updating column name for easy access

df=df.withColumnRenamed("Summons Number", "Summons_Number")
df=df.withColumnRenamed("Plate ID", "Plate_ID")
df=df.withColumnRenamed("Registration State", "Registration_State")
df=df.withColumnRenamed("Issue Date", "Issue_Date")
df=df.withColumnRenamed("Violation Code", "Violation_Code")
df=df.withColumnRenamed("Vehicle Body Type", "Vehicle_Body_Type")
df=df.withColumnRenamed("Vehicle Make", "Vehicle_Make")
df=df.withColumnRenamed("Violation Precinct", "Violation_Precinct")
df=df.withColumnRenamed("Issuer Precinct", "Issuer_Precinct")
df=df.withColumnRenamed("Violation Time", "Violation_Time")

In [7]:
# To check the schema of DF
df.printSchema()

root
 |-- Summons_Number: long (nullable = true)
 |-- Plate_ID: string (nullable = true)
 |-- Registration_State: string (nullable = true)
 |-- Issue_Date: timestamp (nullable = true)
 |-- Violation_Code: integer (nullable = true)
 |-- Vehicle_Body_Type: string (nullable = true)
 |-- Vehicle_Make: string (nullable = true)
 |-- Violation_Precinct: integer (nullable = true)
 |-- Issuer_Precinct: integer (nullable = true)
 |-- Violation_Time: string (nullable = true)



# Part 1: Examine the data

## 1. Find the total number of tickets for the year.

In [8]:
#Total number of records
df.count()

10803028

In [13]:
#Check for Null Values in Each Column. Some columns have null values marked text nan, checking for both cases
df.where(col("Summons_Number") == "nan").count()

0

In [10]:
df.where(col("Summons_Number").isNull()).count()

0

In [83]:
df.where(col("Plate_ID") == "nan").count()

728

In [16]:
df.where(col("Plate_ID").isNull()).count()

0

In [85]:
df.where(col("Registration_State") == "nan").count()

0

In [84]:
df.where(col("Registration_State").isNull()).count()

0

In [86]:
df.where(col("Issue_Date") == "nan").count()

0

In [18]:
df.where(col("Issue_Date").isNull()).count()

0

In [87]:
df.where(col("Violation_Code") == "nan").count()

0

In [19]:
df.where(col("Violation_Code").isNull()).count()

0

In [88]:
df.where(col("Vehicle_Body_Type") == "nan").count()

42711

In [20]:
df.where(col("Vehicle_Body_Type").isNull()).count()

0

In [89]:
df.where(col("Vehicle_Make") == "nan").count()

73050

In [21]:
df.where(col("Vehicle_Make").isNull()).count()

0

In [90]:
df.where(col("Violation_Precinct") == "nan").count()

0

In [22]:
df.where(col("Violation_Precinct").isNull()).count()

0

In [91]:
df.where(col("Issuer_Precinct") == "nan").count()

0

In [23]:
df.where(col("Issuer_Precinct").isNull()).count()

0

In [24]:
df.where(col("Violation_Time").isNull()).count()

0

In [12]:
df.where(col("Violation_Time") == "nan").count()

63

### Thus it is evident that there are a few null values in the dataframe. We will deal with them if necessary in the further analysis

## 2. Find out the number of unique states from where the cars that got parking tickets came. (Hint: Use the column 'Registration State'.)
- There is a numeric entry '99' in the column, which should be corrected. Replace it with the state having the maximum entries. Provide the number of unique states again.

In [29]:
#Number of unique Registration_State

df.select('Registration_State').distinct().count()

67

In [30]:
#Checking count of Registration_State with 99 

df.filter((col("Registration_State") == "99")).count()

36625

In [31]:
# chceking highest number of registation 

from pyspark.sql.functions import count
from pyspark.sql.functions import desc

df.groupBy("Registration_State").agg(count("Registration_State").alias("Num")).sort(desc('Num')).show(10)

+------------------+-------+
|Registration_State|    Num|
+------------------+-------+
|                NY|8481061|
|                NJ| 925965|
|                PA| 285419|
|                FL| 144556|
|                CT| 141088|
|                MA|  85547|
|                IN|  80749|
|                VA|  72626|
|                MD|  61800|
|                NC|  55806|
+------------------+-------+
only showing top 10 rows



In [32]:
#replacing the Registration_State '99' to 'NY' as NY has highest number of registation

from pyspark.sql.functions import *
df = df.withColumn('Registration_State', regexp_replace('Registration_State', '99', 'NY'))

In [33]:
df.filter((col("Registration_State") == "99")).count()

0

In [34]:
#Number of unique Registration_State after replacing 99 with 'NY'

df.select('Registration_State').distinct().count()

66

# Aggregation tasks

## 1. How often does each violation code occur? Display the frequency of the top five violation codes.

In [36]:
#Total disinct Violation Code Present in file..
df.select('Violation_Code').distinct().count()

100

In [37]:
#Top 5 Violation Code Frequency 

df.groupBy("Violation_Code").agg(count("Violation_Code").alias("Frequency")).sort(desc('Frequency')).show(5)

+--------------+---------+
|Violation_Code|Frequency|
+--------------+---------+
|            21|  1528588|
|            36|  1400614|
|            38|  1062304|
|            14|   893498|
|            20|   618593|
+--------------+---------+
only showing top 5 rows



## 2. How often does each 'vehicle body type' get a parking ticket? How about the 'vehicle make'?

In [41]:
#Top 5 Vehicle Body Type with maximum violations

df.groupBy("Vehicle_Body_Type").agg(count("Vehicle_Body_Type").alias("Body_type_top_5")).sort(desc('Body_type_top_5')).show(5)

+-----------------+---------------+
|Vehicle_Body_Type|Body_type_top_5|
+-----------------+---------------+
|             SUBN|        3719802|
|             4DSD|        3082020|
|              VAN|        1411970|
|             DELV|         687330|
|              SDN|         438191|
+-----------------+---------------+
only showing top 5 rows



In [44]:
#Top 5 Vehicle Make with maximum violations

df.groupBy("Vehicle_Make").agg(count("Vehicle_Make").alias("Vehicle_make_top_5")).sort(desc('Vehicle_make_top_5')).show(5)

+------------+------------------+
|Vehicle_Make|Vehicle_make_top_5|
+------------+------------------+
|        FORD|           1280958|
|       TOYOT|           1211451|
|       HONDA|           1079238|
|       NISSA|            918590|
|       CHEVR|            714655|
+------------+------------------+
only showing top 5 rows



### 3. A precinct is a police station that has a certain zone of the city under its command. Find the (5 highest) frequencies of tickets for each of the following:
#### 'Violation Precinct' (This is the precinct of the zone where the violation occurred). Using this, can you draw any insights for parking violations in any specific areas of the city?
##### 'Issuer Precinct' (This is the precinct that issued the ticket.)
#### Here, you would have noticed that the dataframe has the'Violating Precinct' or 'Issuing Precinct' as '0'. These are erroneous entries. Hence, you need to provide the records for five correct precincts. (Hint: Print the top six entries after sorting.)

In [46]:
#Top 5 Violation Precinct. We need to ignore 0 here as it is an incorrect entry

df.groupBy("Violation_Precinct").agg(count("Violation_Precinct").alias("Violation_Precinct_top_5")).sort(desc('Violation_Precinct_top_5')).show(6)

+------------------+------------------------+
|Violation_Precinct|Violation_Precinct_top_5|
+------------------+------------------------+
|                 0|                 2072400|
|                19|                  535671|
|                14|                  352450|
|                 1|                  331810|
|                18|                  306920|
|               114|                  296514|
+------------------+------------------------+
only showing top 6 rows



In [47]:
#Top 5 Issuer Precinct. We need to ignore 0 here as it is an incorrect entry

df.groupBy("Issuer_Precinct").agg(count("Issuer_Precinct").alias("Issuer_Precinct_top_5")).sort(desc('Issuer_Precinct_top_5')).show(6)

+---------------+---------------------+
|Issuer_Precinct|Issuer_Precinct_top_5|
+---------------+---------------------+
|              0|              2388479|
|             19|               521513|
|             14|               344977|
|              1|               321170|
|             18|               296553|
|            114|               289950|
+---------------+---------------------+
only showing top 6 rows



### 4. Find the violation code frequencies for three precincts that have issued the most number of tickets. Do these precinct zones have an exceptionally high frequency of certain violation codes? Are these codes common across precincts?

In [66]:
# Register DataFrame as temp table to Run SQL
df.createOrReplaceTempView("data")

In [49]:
spark.sql("select * from data").show(3)

+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
|Summons_Number|Plate_ID|Registration_State|         Issue_Date|Violation_Code|Vehicle_Body_Type|Vehicle_Make|Violation_Precinct|Issuer_Precinct|Violation_Time|
+--------------+--------+------------------+-------------------+--------------+-----------------+------------+------------------+---------------+--------------+
|    5092469481| GZH7067|                NY|2016-07-10 00:00:00|             7|             SUBN|       TOYOT|                 0|              0|         0143A|
|    5092451658| GZH7067|                NY|2016-07-08 00:00:00|             7|             SUBN|       TOYOT|                 0|              0|         0400P|
|    4006265037| FZX9232|                NY|2016-08-23 00:00:00|             5|             SUBN|        FORD|                 0|              0|         0233P|
+--------------+--------+---------

In [93]:
# Top 3 Violation Code for Precinct 19
spark.sql("select  Violation_Code, count(*) as Precinct_19 from data where Issuer_Precinct = 19 group by Violation_Code order by Precinct_19 desc").show(3)

+--------------+-----------+
|Violation_Code|Precinct_19|
+--------------+-----------+
|            46|      86390|
|            37|      72437|
|            38|      72344|
+--------------+-----------+
only showing top 3 rows



In [94]:
# Top 3 Violation Code for Precinct 14
spark.sql("select  Violation_Code, count(*) as Precinct_14 from data where Issuer_Precinct = 14 group by Violation_Code order by Precinct_14 desc").show(3)

+--------------+-----------+
|Violation_Code|Precinct_14|
+--------------+-----------+
|            14|      73837|
|            69|      58026|
|            31|      39857|
+--------------+-----------+
only showing top 3 rows



In [95]:
# Top 3 Violation Code for Precinct 1
spark.sql("select  Violation_Code, count(*) as Precinct_1 from data where Issuer_Precinct = 1 group by Violation_Code order by Precinct_1 desc").show(3)

+--------------+----------+
|Violation_Code|Precinct_1|
+--------------+----------+
|            14|     73522|
|            16|     38937|
|            20|     27841|
+--------------+----------+
only showing top 3 rows



## 5. Find out the properties of parking violations across different times of the day:
- Find a way to deal with missing values, if any. (Hint: Check for the null values using 'isNull' under the SQL. Also, to remove the null values, check the 'dropna' command in the API documentation.)
- The Violation Time field is specified in a strange format. Find a way to make this a time attribute that you can use to divide into groups.
- Divide 24 hours into six equal discrete bins of time. Choose the intervals as you see fit. For each of these groups, find the three most commonly occurring violations. (Hint: Use the CASE-WHEN in SQL view to segregate into bins. To find the most commonly occurring violations, you can use an approach similar to the one mentioned in the hint for question 4.)
- Now, try another direction. For the three most commonly occurring violation codes, find the most common time of the day (in terms of the bins from the previous part).

In [14]:
# There are missing values in this column, but isnull() is not able to detect them as mentioned earlier,
# the null values in the column are represented by text 'nan'
df.where(col("Violation_Time") == "nan").count()

63

In [15]:
# There are 63 null values which we will remove
df_no_null = df.filter((col("Violation_Time") != "nan"))

In [19]:
# The violation time is in the format HHMM(AM/PM). We will concat M at the end and convert it into datetime format
df_no_null = df_no_null.withColumn("Violation_Time",concat(col("Violation_Time"),lit("M")))

In [21]:
df_no_null = df_no_null.withColumn("Violation_Time",to_timestamp("Violation_Time","hhmma"))

In [23]:
# Register DataFrame as temp table to Run SQL
df_no_null.createOrReplaceTempView("data_q5")

In [56]:
# This provides an overall view of the type of traffic violation per different time of the day
spark.sql("select Violation_Code, count(CASE WHEN HOUR(Violation_Time) >= 0 AND HOUR(Violation_Time) <= 4 THEN 1 END) as Midnight, \
count(CASE WHEN HOUR(Violation_Time) > 4 AND HOUR(Violation_Time) <= 8 THEN 1 END) as Early_Morning, \
count(CASE WHEN HOUR(Violation_Time) > 8 AND HOUR(Violation_Time) <= 12 THEN 1 END) as Morning, \
count(CASE WHEN HOUR(Violation_Time) > 12 AND HOUR(Violation_Time) <= 16 THEN 1 END) as Afternoon, \
count(CASE WHEN HOUR(Violation_Time) > 16 AND HOUR(Violation_Time) <= 20 THEN 1 END) as Evening, \
count(CASE WHEN HOUR(Violation_Time) > 20 THEN 1 END) as Night from data_q5 group by Violation_Code order by Early_Morning desc").show()

+--------------+--------+-------------+-------+---------+-------+-----+
|Violation_Code|Midnight|Early_Morning|Morning|Afternoon|Evening|Night|
+--------------+--------+-------------+-------+---------+-------+-----+
|            21|   55436|       498330| 945652|     4536|    422|  287|
|            14|   26300|       207957| 260338|   279224|  78490|33160|
|            36|       0|       170484| 826311|   394403|   9416|    0|
|            40|   50777|       139381| 128150|   123008|  37348|34688|
|            20|   22746|       129826| 170467|   215704|  53387|22323|
|            71|    8005|        85693| 195524|   190989|  33880| 5919|
|             7|   30162|        62859| 113871|   151209| 112971|45323|
|            46|   13904|        57439| 243282|   209325|  57276|16977|
|            38|     578|        52544| 402653|   453644| 124026|28842|
|            19|   10529|        47441| 102180|    90105|  26308|12462|
|            69|       2|        43708|  69805|    61246|   8709

In [58]:
# Let's check for each of the time periods which are the top violation codes
# Top 5 violation codes for Midnight (12am to 4am)
spark.sql("select Violation_Code, count(CASE WHEN HOUR(Violation_Time) >= 0 AND HOUR(Violation_Time) <= 4 THEN 1 END) as Midnight \
from data_q5 group by Violation_Code order by Midnight desc").show(5)

+--------------+--------+
|Violation_Code|Midnight|
+--------------+--------+
|            21|   55436|
|            40|   50777|
|            78|   31496|
|             7|   30162|
|            14|   26300|
+--------------+--------+
only showing top 5 rows



In [59]:
# Top 5 violation codes for Early Morning (4am to 8am)
spark.sql("select Violation_Code, count(CASE WHEN HOUR(Violation_Time) >= 4 AND HOUR(Violation_Time) <= 8 THEN 1 END) as Early_Morning \
from data_q5 group by Violation_Code order by Early_Morning desc").show(5)

+--------------+-------------+
|Violation_Code|Early_Morning|
+--------------+-------------+
|            21|       500166|
|            14|       210175|
|            36|       170484|
|            40|       145421|
|            20|       132068|
+--------------+-------------+
only showing top 5 rows



In [60]:
# Top 5 violation codes for Morning (8am to 12pm)
spark.sql("select Violation_Code, count(CASE WHEN HOUR(Violation_Time) >= 8 AND HOUR(Violation_Time) <= 12 THEN 1 END) as Morning \
from data_q5 group by Violation_Code order by Morning desc").show(5)

+--------------+-------+
|Violation_Code|Morning|
+--------------+-------+
|            21|1326352|
|            36| 962856|
|            38| 453023|
|            14| 329238|
|            46| 270122|
+--------------+-------+
only showing top 5 rows



In [61]:
# Top 5 violation codes for Afternoon (12pm to 4pm)
spark.sql("select Violation_Code, count(CASE WHEN HOUR(Violation_Time) >= 12 AND HOUR(Violation_Time) <= 16 THEN 1 END) as Afternoon \
from data_q5 group by Violation_Code order by Afternoon desc").show(5)

+--------------+---------+
|Violation_Code|Afternoon|
+--------------+---------+
|            36|   605837|
|            38|   560149|
|            37|   410966|
|            14|   334178|
|            46|   265753|
+--------------+---------+
only showing top 5 rows



In [62]:
# Top 5 violation codes for Evening (4pm to 8pm)
spark.sql("select Violation_Code, count(CASE WHEN HOUR(Violation_Time) >= 16 AND HOUR(Violation_Time) <= 20 THEN 1 END) as Evening \
from data_q5 group by Violation_Code order by Evening desc").show(5)

+--------------+-------+
|Violation_Code|Evening|
+--------------+-------+
|            38| 221419|
|            14| 156366|
|             7| 152038|
|            37| 151692|
|            46|  93704|
+--------------+-------+
only showing top 5 rows



In [63]:
# Top 5 violation codes for Night (8pm to 12am)
spark.sql("select Violation_Code, count(CASE WHEN HOUR(Violation_Time) >= 20 THEN 1 END) as Night \
from data_q5 group by Violation_Code order by Night desc").show(5)

+--------------+-----+
|Violation_Code|Night|
+--------------+-----+
|             7|65593|
|            38|47029|
|            14|44778|
|            40|44541|
|            20|31084|
+--------------+-----+
only showing top 5 rows



### The three most commonly occuring violations are 14, 38 and 46

### For these three violations let's find the most common time of day

In [64]:
spark.sql("select count(CASE WHEN HOUR(Violation_Time) >= 0 AND HOUR(Violation_Time) <= 4 THEN 1 END) as Midnight, \
count(CASE WHEN HOUR(Violation_Time) > 4 AND HOUR(Violation_Time) <= 8 THEN 1 END) as Early_Morning, \
count(CASE WHEN HOUR(Violation_Time) > 8 AND HOUR(Violation_Time) <= 12 THEN 1 END) as Morning, \
count(CASE WHEN HOUR(Violation_Time) > 12 AND HOUR(Violation_Time) <= 16 THEN 1 END) as Afternoon, \
count(CASE WHEN HOUR(Violation_Time) > 16 AND HOUR(Violation_Time) <= 20 THEN 1 END) as Evening, \
count(CASE WHEN HOUR(Violation_Time) > 20 THEN 1 END) as Night from data_q5 where Violation_Code in (14,38,46)").show()

+--------+-------------+-------+---------+-------+-----+
|Midnight|Early_Morning|Morning|Afternoon|Evening|Night|
+--------+-------------+-------+---------+-------+-----+
|   40782|       317940| 906273|   942193| 259792|78979|
+--------+-------------+-------+---------+-------+-----+



### Thus for the top three violations Mornings and Afternoons are the most common

## 6. Let’s try and find some seasonality in this data:

- First, divide the year into a certain number of seasons, and find the frequencies of tickets for each season. (Hint: Use Issue Date to segregate into seasons.)

- Then, find the three most common violations for each of these seasons.
(Hint: You can use an approach similar to the one mentioned in the hint for question 4.)

In [69]:
# Let's categorize the data into seasons and take count of tickets for each season
spark.sql("select count(CASE WHEN MONTH(Issue_Date) >= 3 AND MONTH(Issue_Date) < 6 THEN 1 END) as Spring, \
count(CASE WHEN MONTH(Issue_Date) >= 6 AND MONTH(Issue_Date) < 9 THEN 1 END) as Summer, \
count(CASE WHEN MONTH(Issue_Date) >= 9 AND MONTH(Issue_Date) < 12 THEN 1 END) as Autumn, \
count(CASE WHEN MONTH(Issue_Date) = 12 OR MONTH(Issue_Date) < 3 THEN 1 END) as Winter from data").show()

+-------+-------+-------+-------+
| Spring| Summer| Autumn| Winter|
+-------+-------+-------+-------+
|2880687|2606208|2830802|2485331|
+-------+-------+-------+-------+



In [ ]:
# Let's look at the top 3 violations of each of these seasons

In [72]:
# Top 3 violations for Spring
spark.sql("select Violation_Code, count(CASE WHEN MONTH(Issue_Date) >= 3 AND MONTH(Issue_Date) < 6 THEN 1 END) as Spring \
from data group by Violation_Code order by Spring desc ").show(3)

+--------------+------+
|Violation_Code|Spring|
+--------------+------+
|            21|402807|
|            36|344834|
|            38|271192|
+--------------+------+
only showing top 3 rows



In [73]:
# Top 3 violations for Summer
spark.sql("select Violation_Code, count(CASE WHEN MONTH(Issue_Date) >= 6 AND MONTH(Issue_Date) < 9 THEN 1 END) as Summer \
from data group by Violation_Code order by Summer desc ").show(3)

+--------------+------+
|Violation_Code|Summer|
+--------------+------+
|            21|405961|
|            38|247561|
|            36|240396|
+--------------+------+
only showing top 3 rows



In [74]:
# Top 3 violations for Autumn
spark.sql("select Violation_Code, count(CASE WHEN MONTH(Issue_Date) >= 9 AND MONTH(Issue_Date) < 12 THEN 1 END) as Autumn \
from data group by Violation_Code order by Autumn desc ").show(3)

+--------------+------+
|Violation_Code|Autumn|
+--------------+------+
|            36|456046|
|            21|357479|
|            38|283828|
+--------------+------+
only showing top 3 rows



In [75]:
# Top 3 violations for Winter
spark.sql("select Violation_Code, count(CASE WHEN MONTH(Issue_Date) == 12 AND MONTH(Issue_Date) < 3 THEN 1 END) as Winter \
from data group by Violation_Code order by Winter desc ").show(3)

+--------------+------+
|Violation_Code|Winter|
+--------------+------+
|            85|     0|
|            53|     0|
|            65|     0|
+--------------+------+
only showing top 3 rows



## 7. The fines collected from all the instances of parking violation constitute a source of revenue for the NYC Police Department. Let’s take an example of estimating this for the three most commonly occurring codes:
- Find the total occurrences of the three most common violation codes.
- Then, visit the website: http://www1.nyc.gov/site/finance/vehicles/services-violation-codes.page
- It lists the fines associated with different violation codes. They’re divided into two categories: one for the highest-density locations in the city and the other for the rest of the city. For the sake of simplicity, take the average of the two.
- Using this information, find the total amount collected for the three violation codes with the maximum tickets. State the code that has the highest total collection.
- What can you intuitively infer from these findings?

In [79]:
#Let's find the three most common violation codes

df.groupBy("Violation_Code").agg(count("Violation_Code").alias("Frequency")).sort(desc('Frequency')).show(3)

+--------------+---------+
|Violation_Code|Frequency|
+--------------+---------+
|            21|  1528588|
|            36|  1400614|
|            38|  1062304|
+--------------+---------+
only showing top 3 rows



In [ ]:
# Fines for Violation Codes
# 21 - Street Cleaning: No parking where parking is not allowed by sign, street marking or traffic control device - 55$
# 36 - Exceeding the posted speed limit in or near a designated school zone - 50$
# 38 - Failing to show a receipt or tag in the windshield - 50$

In [80]:
# Finding collection for each of these violation codes
# Collection for Violation Code 21
df.filter((col("Violation_Code") == "21")).count()* 55

84072340

In [81]:
# Collection for Violation Code 36
df.filter((col("Violation_Code") == "36")).count()* 50

70030700

In [82]:
# Collection for Violation Code 38
df.filter((col("Violation_Code") == "38")).count()* 50

53115200

### Thus the Violation Code 21 has maximum collection as well as count. Evident from the description as well, it is the most common traffic violation and with its high fine, it is the most productive violation 